In [216]:
import pandas as pd

In [217]:
pkl='allyears_with_teams'
df= pd.read_pickle(pkl)

In [218]:
#Create lists of players on each team
with open('euro_players_scrape.txt', 'r') as rf:
    euro_players=[]
    for line in rf:
        n=line.split('  ')
        euro_players.append(n[0])
with open('am_players_scrape.txt', 'r') as rf:
    am_players=[]
    for line in rf:
        n=line.split('  ')
        am_players.append(n[0])

In [219]:
#remove non-playing captains and fix oscar's name
am_players.remove('Ã“scar DomÃ\xadnguez')
am_players.append('Óscar Domínguez')
am_players.remove('Buddy Hall')
am_players.remove('Don Wardell ')
euro_players.remove('Johan Ruijsink')

In [220]:
players = am_players + euro_players

In [221]:
#create player dictionary to boolean index df by player
#*note: str.contains allows matches for when there are other characters as well, such as in doubles 
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [222]:
#Create function for boolean indexing the dataframe
def boolind(df = df, d = d,**kwargs):
    if 'form' in kwargs.keys():
        df = df[df['Format']==kwargs['form']]
    if 'Europe_won' in kwargs.keys():
        df = df [df['Europe_won']==kwargs['Europe_won']]
    if 'player' in kwargs.keys():
        df = df [d [  kwargs['player']  ] ]
    if 'year' in kwargs.keys():
        df = df.loc[kwargs['year']]
    return(df)

In [223]:
boolind(form='Teams',year=2014)['European_player']

Match
0    Karl Boyes, Niels Feijen, Nikos Ekonomopoulos,...
Name: European_player, dtype: object

In [224]:
df.loc[2014]

,Format,European_player,European_score,American_score,American_player,Europe_won
Match,,,,,,
0,Teams,"Karl Boyes, Niels Feijen, Nikos Ekonomopoulos,...",5,1,"Corey Deuel, Shane Van Boening, Justin Bergman...",True
1,Doubles,Darren Appleton & Karl Boyes,3,5,Corey Deuel & Justin Hall,False
2,Singles,Mark Gray,3,5,John Schmidt,False
3,Doubles,Niels Feijen & Nikos Ekonomopoulos,5,1,Shane Van Boening & Justin Bergman,True
4,Singles,Darren Appleton,5,3,Shane Van Boening,True
5,Doubles,Niels Feijen & Mark Gray,2,5,Justin Hall & Justin Bergman,False
6,Singles,Karl Boyes,5,2,Corey Deuel,True
7,Doubles,Darren Appleton & Nikos Ekonomopoulos,5,2,Shane Van Boening & John Schmidt,True
8,Singles,Niels Feijen,5,3,Justin Bergman,True


In [225]:
# def wl(**kwargs):
#     '''
#     kwargs: player, country, format, year, Europe_won
#     '''
#     lst= 'player, country, format, year, Europe_won'.split()
#     print(lst)
#     for item in lst:
#         if item in kwargs.keys():
#             item=
#     if form in kwargs.keys():
#         form=kwargs['form']
        
#     if 'a' in country:
#         win=df['Europe_won']==False].shape[0]
#         loss=
#     else:
#         loss=boolind(form=form,player=player,Europe_won=False).shape[0]
#         win=boolind(form=form,player=player,Europe_won=True).shape[0]      
#     swl='Win-Loss: {}-{}'.format(win,loss)
#     spct=round(win*100/(win+loss),2)
#     return(swl, 'Pct: {}'.format(spct))

In [226]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [227]:
def swl(player,country):
    #singles
    if 'a' in country:
        win=df[america_won&singles&d[player]].shape[0]
        loss=df[d[player]&~america_won&singles].shape[0]
    else:
        loss=df[d[player]&america_won&singles].shape[0]
        win=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(win,loss)
    spct=round(win*100/(win+loss),2)
    return(swl, spct)


def dwl(player,country):
    if 'a' in country:
        win=df[d[player]&america_won&doubles].shape[0]
        loss=df[d[player]&~america_won&doubles].shape[0]
    else:
        loss=df[d[player]&america_won&doubles].shape[0]
        win=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(win,loss)
    return(dwl)


def cwl(player,country):
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        swin=df[d[player]&america_won&singles].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0]
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]
    win=dwin+swin
    loss=dloss+sloss
    cwl='{}-{}'.format(win,loss)
    return(cwl)

In [228]:
def winloss(player, country):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(win,loss)
    spct=win*100/(win+loss)

    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(win,loss)
    dpct=win*100/(win+loss)
    
    #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
    twl='{}-{}'.format(win,loss)
    tpct=win*100/(win+loss)
    
    #combined
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        swin=df[d[player]&america_won&singles].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0]
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]
    win=dwin+swin
    loss=dloss+sloss
    cwl='{}-{}'.format(win,loss)
    cpct=win*100/(win+loss)
    print("singles: {}   pct:{}   doubles: {}  pct:{}    combined: {}  pct:{}".format(swl,spct,dwl,dpct,cwl,cpct))
    return(swl,dwl,cwl)

In [229]:
# def wlfcn(player, country):
#     match = df['Format']==form
#     #singles
#     if 'a' in country:
#         win=df[america_won&match&d[player]].shape[0]
#         loss=df[d[player]&~america_won&singles].shape[0]
#     else:
#         loss=df[d[player]&america_won&singles].shape[0]
#         win=df[d[player]&~america_won&singles].shape[0]        
#     swl='{}-{}'.format(win,loss)
#     spct=win*100/(win+loss)

#     #doubles
#     if 'a' in country:
#         win=df[d[player]&america_won&doubles].shape[0]
#         loss=df[d[player]&~america_won&doubles].shape[0]
#     else:
#         loss=df[d[player]&america_won&doubles].shape[0]
#         win=df[d[player]&~america_won&doubles].shape[0] 
#     dwl='{}-{}'.format(win,loss)
#     dpct=win*100/(win+loss)

#     #combined
#     if 'a' in country:
#         dwin=df[d[player]&america_won&doubles].shape[0]
#         dloss=df[d[player]&~america_won&doubles].shape[0]
#         swin=df[d[player]&america_won&singles].shape[0]
#         sloss=df[d[player]&~america_won&singles].shape[0]
#     else:
#         dloss=df[d[player]&america_won&doubles].shape[0]
#         dwin=df[d[player]&~america_won&doubles].shape[0]
#         sloss=df[d[player]&america_won&singles].shape[0]
#         swin=df[d[player]&~america_won&singles].shape[0]
#     win=dwin+swin
#     loss=dloss+sloss
#     cwl='{}-{}'.format(win,loss)
#     cpct=win*100/(win+loss)
#     print("singles: {}   pct:{}   doubles: {}  pct:{}    combined: {}  pct:{}".format(swl,spct,dwl,dpct,cwl,cpct))
#     return(swl,dwl,cwl)

In [230]:
def winlosslist(player, country):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(swin,sloss)
    try:
        spct=swin*100/(swin+sloss)
    except:
        pass
    
    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(dwin,dloss)
    try:
        dpct=dwin*100/(dwin+dloss)   
    except:
        dpct=0
    
        #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
    twl='{}-{}'.format(twin,tloss)
    try:
        tpct=twin*100/(twin+tloss)   
    except:
        tpct=0

    #combined
    cwin=dwin+swin+twin
    closs=dloss+sloss+tloss
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,swin,sloss,swl,round(spct,2),dwin,dloss,dwl,round(dpct,2), twl, cwin,closs,cwl,round(cpct,2)]
    return(l)

In [231]:
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        print('failure- am: ', player)
#         pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        print('failure -eu ', player)
#         pass

In [232]:
import pandas as pd
colz = 'Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Twl,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)
player_stats.sort_values('Cpct',axis=0, ascending=False)

,Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Twl,Cwin,Closs,Cwl,Cpct
38,Vivian Villarreal,America,3,0,3-0,100.00,2,0,2-0,100.00,0-0,5,0,5-0,100.00
35,Jeanette Lee,America,2,0,2-0,100.00,3,0,3-0,100.00,0-0,5,0,5-0,100.00
54,Albin Ouschan,Europe,3,0,3-0,100.00,5,1,5-1,83.33,1-1,9,2,9-2,81.82
36,Gabe Owen,America,2,0,2-0,100.00,2,1,2-1,66.67,0-0,4,1,4-1,80.00
26,C.J. Wiley,America,3,1,3-1,75.00,1,0,1-0,100.00,0-0,4,1,4-1,80.00
48,Karl Boyes,Europe,4,2,4-2,66.67,10,2,10-2,83.33,3-1,17,5,17-5,77.27
16,Shannon Daulton,America,5,1,5-1,83.33,2,2,2-2,50.00,0-0,7,3,7-3,70.00
27,Mike Davis,America,1,1,1-1,50.00,1,1,1-1,50.00,2-0,4,2,4-2,66.67
42,Niels Feijen,Europe,12,8,12-8,60.00,24,9,24-9,72.73,5-4,41,21,41-21,66.13
60,Tom Storm,Europe,5,3,5-3,62.50,2,1,2-1,66.67,0-0,7,4,7-4,63.64


## Mosconi Cup Trivia Questions
Most wins? (s,d,c)
Highest win %? (s,d,c)
Lowest win %? (s,d,c)


In [233]:
def playerinfo(player,country):
    '''Get list of info about player stats and output every match they played'''
    wl=winlosslist(player,country)
    frame=df[d[player]]
    print((winlosslist(player,country)))
    return(frame)

In [234]:
_=playerinfo('Niels Feijen','f')
_[_['Format']=='Singles']

['Niels Feijen', 'f', 12, 8, '12-8', 60.0, 24, 9, '24-9', 72.73, '5-4', 41, 21, '41-21', 66.13]


Format European_player European_score American_score  \
Year Match                                                          
2001 10     Singles    Niels Feijen              2              5   
2004 7      Singles    Niels Feijen              1              5   
     16     Singles    Niels Feijen              5              2   
2005 3      Singles    Niels Feijen              3              5   
2007 4      Singles    Niels Feijen              1              6   
     13     Singles    Niels Feijen              6              4   
2008 11     Singles    Niels Feijen              3              5   
2009 11     Singles    Niels Feijen              4              6   
     15     Singles    Niels Feijen              0              6   
2011 2      Singles    Niels Feijen              6              0   
     17     Singles    Niels Feijen              6              4   
2012 2      Singles    Niels Feijen              5              0   
     16     Singles    Niels Feijen              1              5   
2013 8      Singles    Niels Feijen              6              0   
2014 8      Singles    Niels Feijen              5              3   
     13     Singles    Niels Feijen              5              2   
2015 2      Singles    Niels Feijen              5              2   
     13     Singles    Niels Feijen              5              4   
     15     Singles    Niels Feijen              5              4   
2016 2      Singles    Niels Feijen              5              2   

              American_player Europe_won  
Year Match                                
2001 10       Earl Strickland      False  
2004 7       Charlie Williams      False  
     16         Johnny Archer       True  
2005 3       Charlie Williams      False  
2007 4          Rodney Morris      False  
     13         Rodney Morris       True  
2008 11         Rodney Morris      False  
2009 11          Dennis Hatch      False  
     15          Dennis Hatch      False  
2011 2          Rodney Morris       True  
     17         Rodney Morris       True  
2012 2          Mike Dechaine       True  
     16         Johnny Archer      False  
2013 8          Johnny Archer       True  
2014 8         Justin Bergman       True  
     13        Justin Bergman       True  
2015 2         Justin Bergman       True  
     13     Shane Van Boening       True  
     15        Justin Bergman       True  
2016 2          Mike Dechaine       True

In [235]:
stat_l = []
for player in am_players:
    try:
        unit = winlosslist(player,'America')
        stat_l.append(unit)
    except:
        pass
for player in euro_players:
    try:
        unit = winlosslist(player,'Europe')
        stat_l.append(unit)
    except:
        pass

In [236]:
stat_cols = ['player','team','s_wins','s_loss', 's_winloss', 's_pct','d_wins','d_loss', 'd_winloss', 'd_pct','t_winloss','c_wins','c_loss', 'c_winloss', 'c_pct']

In [237]:
#Most combined wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','c_winloss','c_pct','c_wins']].sort_values('c_wins',ascending=False).reset_index(drop=True)

,player,c_winloss,c_pct,c_wins
0,Ralf Souquet,51-34,60.00,51
1,Johnny Archer,49-40,55.06,49
2,Mika Immonen,41-37,52.56,41
3,Earl Strickland,41-26,61.19,41
4,Niels Feijen,41-21,66.13,41
5,Darren Appleton,25-22,53.19,25
6,Shane Van Boening,25-35,41.67,25
7,Rodney Morris,25-30,45.45,25
8,Corey Deuel,25-20,55.56,25
9,Steve Davis,22-30,42.31,22


In [238]:
#Best combined win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','c_winloss','c_pct']].sort_values('c_pct',ascending=False).head(20).reset_index(drop=True)

,player,c_winloss,c_pct
0,Vivian Villarreal,5-0,100.00
1,Jeanette Lee,5-0,100.00
2,Albin Ouschan,9-2,81.82
3,Gabe Owen,4-1,80.00
4,C.J. Wiley,4-1,80.00
5,Karl Boyes,17-5,77.27
6,Shannon Daulton,7-3,70.00
7,Mike Davis,4-2,66.67
8,Niels Feijen,41-21,66.13
9,Tom Storm,7-4,63.64


In [239]:
#Most singles wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','s_winloss','s_wins']].sort_values('s_wins',ascending=False).head(11).reset_index(drop=True)

,player,s_winloss,s_wins
0,Ralf Souquet,20-13,20
1,Johnny Archer,17-13,17
2,Mika Immonen,16-17,16
3,Steve Davis,14-13,14
4,Earl Strickland,14-12,14
5,Niels Feijen,12-8,12
6,Oliver Ortmann,11-10,11
7,Rodney Morris,10-8,10
8,Darren Appleton,10-4,10
9,Shane Van Boening,10-15,10


In [240]:
playerinfo('Justin Hall','am')

['Justin Hall', 'am', 1, 0, '1-0', 100.0, 2, 1, '2-1', 66.67, '0-1', 3, 2, '3-2', 60.0]


Format                                    European_player  \
Year Match                                                               
2014 0        Teams  Karl Boyes, Niels Feijen, Nikos Ekonomopoulos,...   
     1      Doubles                       Darren Appleton & Karl Boyes   
     5      Doubles                           Niels Feijen & Mark Gray   
     10     Doubles                   Nikos Ekonomopoulos & Karl Boyes   
     11     Singles                                Nikos Ekonomopoulos   

           European_score American_score  \
Year Match                                 
2014 0                  5              1   
     1                  3              5   
     5                  2              5   
     10                 5              3   
     11                 3              5   

                                              American_player Europe_won  
Year Match                                                                
2014 0      Corey Deuel, Shane Van Boening, Justin Bergman...       True  
     1                              Corey Deuel & Justin Hall      False  
     5                           Justin Hall & Justin Bergman      False  
     10                            John Schmidt & Justin Hall       True  
     11                                           Justin Hall      False

In [241]:
playerinfo('Justin Bergman','am')

['Justin Bergman', 'am', 1, 5, '1-5', 16.67, 4, 3, '4-3', 57.14, '1-2', 6, 10, '6-10', 37.5]


Format                                    European_player  \
Year Match                                                               
2014 0        Teams  Karl Boyes, Niels Feijen, Nikos Ekonomopoulos,...   
     3      Doubles                 Niels Feijen & Nikos Ekonomopoulos   
     5      Doubles                           Niels Feijen & Mark Gray   
     8      Singles                                       Niels Feijen   
     13     Singles                                       Niels Feijen   
2015 0        Teams  Karl Boyes, Niels Feijen, Albin Ouschan, Nick ...   
     2      Singles                                       Niels Feijen   
     5      Doubles                     Niels Feijen Nick van den Berg   
     10     Doubles                         Niels Feijen Albin Ouschan   
     14     Singles                                    Darren Appleton   
     15     Singles                                       Niels Feijen   
2016 0        Teams  Niels Feijen, Albin Ouschan, Jayson Shaw, Mark...   
     3      Doubles                        Jayson Shaw Darren Appleton   
     7      Doubles                      Albin Ouschan Darren Appleton   
     8      Singles                                          Mark Gray   
     12     Doubles                          Mark Gray Darren Appleton   

           European_score American_score  \
Year Match                                 
2014 0                  5              1   
     3                  5              1   
     5                  2              5   
     8                  5              3   
     13                 5              2   
2015 0                  4              5   
     2                  5              2   
     5                  3              5   
     10                 5              1   
     14                 2              5   
     15                 5              4   
2016 0                  5              2   
     3                  1              5   
     7                  5              4   
     8                  5              1   
     12                 4              5   

                                              American_player Europe_won  
Year Match                                                                
2014 0      Corey Deuel, Shane Van Boening, Justin Bergman...       True  
     3                     Shane Van Boening & Justin Bergman       True  
     5                           Justin Hall & Justin Bergman      False  
     8                                         Justin Bergman       True  
     13                                        Justin Bergman       True  
2015 0      Corey Deuel, Mike Dechaine, Justin Bergman, Sh...      False  
     2                                         Justin Bergman       True  
     5                         Justin Bergman Skyler Woodward      False  
     10                            Justin Bergman Corey Deuel       True  
     14                                        Justin Bergman      False  
     15                                        Justin Bergman       True  
2016 0      Mike Dechaine, Shane Van Boening, Rodney Morri...       True  
     3                         Skyler Woodward Justin Bergman      False  
     7                           Mike Dechaine Justin Bergman       True  
     8                                         Justin Bergman       True  
     12                          Justin Bergman Rodney Morris      False

In [242]:
stats_df.head()

,player,team,s_wins,s_loss,s_winloss,s_pct,d_wins,d_loss,d_winloss,d_pct,t_winloss,c_wins,c_loss,c_winloss,c_pct
0,Johnny Archer,America,17,13,17-13,56.67,27,23,27-23,54.00,5-4,49,40,49-40,55.06
1,Earl Strickland,America,14,12,14-12,53.85,24,12,24-12,66.67,3-2,41,26,41-26,61.19
2,Rodney Morris,America,10,8,10-8,55.56,12,17,12-17,41.38,3-5,25,30,25-30,45.45
3,Shane Van Boening,America,10,15,10-15,40.00,11,14,11-14,44.00,4-6,25,35,25-35,41.67
4,Corey Deuel,America,7,6,7-6,53.85,13,12,13-12,52.00,5-2,25,20,25-20,55.56


In [243]:
#Best singles win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','s_wins','s_winloss','s_pct']].sort_values(['s_pct','s_wins'],ascending=[False,False]).head(28).reset_index(drop=True)
# df.sort_values(['a', 'b'], ascending=[True, False])

,player,s_wins,s_winloss,s_pct
0,Chris Melling,5,5-0,100.00
1,Reed Pierce,4,4-0,100.00
2,Vivian Villarreal,3,3-0,100.00
3,Albin Ouschan,3,3-0,100.00
4,Jeanette Lee,2,2-0,100.00
5,Gabe Owen,2,2-0,100.00
6,Robert Hunter,1,1-0,100.00
7,Justin Hall,1,1-0,100.00
8,Raj Hundal,1,1-0,100.00
9,Kim Davenport,5,5-1,83.33


In [244]:
#Most doubles wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','d_winloss','d_wins']].sort_values('d_wins',ascending=False).reset_index(drop=True).head(10)

,player,d_winloss,d_wins
0,Johnny Archer,27-23,27
1,Ralf Souquet,27-17,27
2,Earl Strickland,24-12,24
3,Niels Feijen,24-9,24
4,Mika Immonen,22-17,22
5,Corey Deuel,13-12,13
6,Rodney Morris,12-17,12
7,Shane Van Boening,11-14,11
8,Jeremy Jones,10-9,10
9,Darren Appleton,10-15,10


In [245]:
#Best doubles win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','d_winloss','d_pct']].sort_values('d_pct',ascending=False).head(20).reset_index(drop=True)

,player,d_winloss,d_pct
0,Vivian Villarreal,2-0,100.00
1,Dallas West,3-0,100.00
2,Bobby Hunter,3-0,100.00
3,Jeanette Lee,3-0,100.00
4,Lou Butera,2-0,100.00
5,Mark Wilson,2-0,100.00
6,C.J. Wiley,1-0,100.00
7,Vincent Facquet,1-0,100.00
8,Karl Boyes,10-2,83.33
9,Albin Ouschan,5-1,83.33


Data including 2016

Team USA
• Shane Van Boening (10 years -- 2007-2016) ..... 10-15 singles, 11-14 doubles, 4-6 teams, 25-35 total (winning percentage of 42%)

• Rodney Morris (10 years -- 2003-2008, 2010-2011, 2013, 2016) ..... 10-8 singles, 12-17 doubles, 2-0 trebles, 3-5 teams, 27-30 total (47%)

• Mike Dechaine (4 years -- 2011-2012, 2015-2016) ..... 1-6 singles, 2-8 doubles, 2-2 teams, 5-16 total (24%)

• Justin Bergman (3 years -- 2014-2016) ..... 1-5 singles, 4-3 doubles, 1-2 teams, 6-10 total (38%)

• Skyler Woodward (2 years -- 2015-2016) ..... 0-3 singles, 3-3 doubles, 1-1 teams, 4-7 total (36%)

• Team USA Total ..... 22-37 singles, 32-45 doubles, 2-0 trebles, 11-16 teams, 67-98 total (41%)

Team Europe
• Niels Feijen (12 years -- 2001, 2004-2005, 2007-2009, 2011-2016) ..... 12-8 singles, 24-9 doubles, 5-4 teams, 41-21 total (66%)

• Darren Appleton (8 years -- 2009-2016) ..... 10-4 singles, 10-15 doubles, 5-3 teams, 25-22 total (53%)

• Mark Gray (3 years -- 2008, 2014, 2016) ..... 1-1 singles, 4-5 doubles, 3-0 teams, 8-6 total (57%)

• Albin Ouschan (2 years -- 2015-2016) ..... 3-0 singles, 5-1 doubles, 1-1 teams, 9-2 total (82%)

• Jayson Shaw (1 year -- 2016) ..... 1-1 singles, 1-1 doubles, 1-0 teams, 3-2 total (60%)

• Team Europe Total ..... 27-14 singles, 44-31 doubles, 15-8 teams, 86-53 total (62%)

In [246]:
df[df['Format']=='Teams']

Format                                    European_player  \
Year Match                                                             
2006 0      Teams  Nick van den Berg, David Alcaide, Thomas Enger...   
     15     Teams  Nick van den Berg, David Alcaide, Thomas Enger...   
2007 0      Teams  Daryl Peach, Tony Drago, Konstantin Stepanov, ...   
2008 0      Teams  Niels Feijen, Tony Drago, Mika Immonen, Ralf S...   
2009 0      Teams  Niels Feijen, Darren Appleton, Mika Immonen, T...   
2010 0      Teams  Nick van den Berg, Karl Boyes, Darren Appleton...   
2011 0      Teams  Nick van den Berg, Niels Feijen, Darren Applet...   
2012 0      Teams  Nick van den Berg, Niels Feijen, Nick Ekonomop...   
2013 0      Teams  Karl Boyes, Niels Feijen, Ralf Souquet, Mika I...   
2014 0      Teams  Karl Boyes, Niels Feijen, Nikos Ekonomopoulos,...   
2015 0      Teams  Karl Boyes, Niels Feijen, Albin Ouschan, Nick ...   
2016 0      Teams  Niels Feijen, Albin Ouschan, Jayson Shaw, Mark...   

           European_score American_score  \
Year Match                                 
2006 0                  5              6   
     15                 3              6   
2007 0                  4              6   
2008 0                  5              3   
2009 0                  4              6   
2010 0                  6              5   
2011 0                  6              3   
2012 0                  3              5   
2013 0                  6              3   
2014 0                  5              1   
2015 0                  4              5   
2016 0                  5              2   

                                              American_player Europe_won  
Year Match                                                                
2006 0      Rodney Morris, John Schmidt, Corey Deuel, Earl...      False  
     15     Rodney Morris, John Schmidt, Corey Deuel, Earl...      False  
2007 0      Corey Deuel, Earl Strickland, Shane Van Boenin...      False  
2008 0      Earl Strickland, Jeremy Jones, Rodney Morris, ...       True  
2009 0      Corey Deuel, Óscar Domínguez, Shane Van Boenin...      False  
2010 0      Corey Deuel, Shane Van Boening, Rodney Morris,...       True  
2011 0      Shawn Putnam, Mike Dechaine, Shane Van Boening...       True  
2012 0      Dennis Hatch, Mike Dechaine, Brandon Shuff, Sh...      False  
2013 0      Dennis Hatch, Earl Strickland, Shane Van Boeni...       True  
2014 0      Corey Deuel, Shane Van Boening, Justin Bergman...       True  
2015 0      Corey Deuel, Mike Dechaine, Justin Bergman, Sh...      False  
2016 0      Mike Dechaine, Shane Van Boening, Rodney Morri...       True

Team USA
• Shane Van Boening 10-15 singles, 11-14 doubles, 4-6 teams, 25-35 total (42%)

• Rodney Morris 10-8 singles, 12-17 doubles, 2-0 trebles, 3-5 teams, 27-30 total (47%)

• Mike Dechaine 1-6 singles, 2-8 doubles, 2-2 teams, 5-16 total (24%)

• Justin Bergman  1-5 singles, 4-3 doubles, 1-2 teams, 6-10 total (38%)

• Skyler Woodward  0-3 singles, 3-3 doubles, 1-1 teams, 4-7 total (36%)


Team Europe
• Niels Feijen 12-8 singles, 24-9 doubles, 5-4 teams, 41-21 total (66%)

• Darren Appleton  10-4 singles, 10-15 doubles, 5-3 teams, 25-22 total (53%)

• Mark Gray   1-1 singles, 4-5 doubles, 3-0 teams, 8-6 total (57%)

• Albin Ouschan  3-0 singles, 5-1 doubles, 1-1 teams, 9-2 total (82%)

• Jayson Shaw 1-1 singles, 1-1 doubles, 1-0 teams, 3-2 total (60%)

In [247]:
usa = ['Shane Van Boening', 'Rodney Morris', 'Mike Dechaine', 'Justin Bergman', 'Skyler Woodward']
for player in usa:
    print(winlosslist(player, 'am'))

['Shane Van Boening', 'am', 10, 15, '10-15', 40.0, 11, 14, '11-14', 44.0, '4-6', 25, 35, '25-35', 41.67]
['Rodney Morris', 'am', 10, 8, '10-8', 55.56, 12, 17, '12-17', 41.38, '3-5', 25, 30, '25-30', 45.45]
['Mike Dechaine', 'am', 1, 6, '1-6', 14.29, 2, 8, '2-8', 20.0, '2-2', 5, 16, '5-16', 23.81]
['Justin Bergman', 'am', 1, 5, '1-5', 16.67, 4, 3, '4-3', 57.14, '1-2', 6, 10, '6-10', 37.5]
['Skyler Woodward', 'am', 0, 3, '0-3', 0.0, 3, 3, '3-3', 50.0, '1-1', 4, 7, '4-7', 36.36]


In [248]:
eu = ['Darren Appleton', 'Mark Gray', 'Albin Ouschan', 'Niels Feijen']
for player in eu:
    print(winlosslist(player, 'u'))

['Darren Appleton', 'u', 10, 4, '10-4', 71.43, 10, 15, '10-15', 40.0, '5-3', 25, 22, '25-22', 53.19]
['Mark Gray', 'u', 1, 1, '1-1', 50.0, 4, 5, '4-5', 44.44, '3-0', 8, 6, '8-6', 57.14]
['Albin Ouschan', 'u', 3, 0, '3-0', 100.0, 5, 1, '5-1', 83.33, '1-1', 9, 2, '9-2', 81.82]
['Niels Feijen', 'u', 12, 8, '12-8', 60.0, 24, 9, '24-9', 72.73, '5-4', 41, 21, '41-21', 66.13]


#More info to verify:
• Johnny Archer (17 years) ..... 18-13 singles, 26-23 doubles, 0-2 trebles, 5-4 teams, 49-42 total (winning percentage of 54%)
• Earl Strickland (14 years) ..... 14-12 singles, 24-12 doubles, 1-1 trebles, 3-2 teams, 42-27 total (61%)
• Corey Deuel (9 years) ..... 7-6 singles, 13-12 doubles, 2-0 trebles, 5-2 teams, 27-20 total (57%)
And more:
Dennis Hatch (4 years -- 2009, 2010, 2012, 2013) ..... 4-3 singles, 5-7 doubles, 2-2 teams, 11-12 total (winning percentage 48%)

In [249]:
pl=['Johnny Archer','Earl Strickland','Corey Deuel','Dennis Hatch']
for player in pl:
    print(winlosslist(player,'am'))

['Johnny Archer', 'am', 17, 13, '17-13', 56.67, 27, 23, '27-23', 54.0, '5-4', 49, 40, '49-40', 55.06]
['Earl Strickland', 'am', 14, 12, '14-12', 53.85, 24, 12, '24-12', 66.67, '3-2', 41, 26, '41-26', 61.19]
['Corey Deuel', 'am', 7, 6, '7-6', 53.85, 13, 12, '13-12', 52.0, '5-2', 25, 20, '25-20', 55.56]
['Dennis Hatch', 'am', 4, 3, '4-3', 57.14, 5, 7, '5-7', 41.67, '2-2', 11, 12, '11-12', 47.83]


Johnny's info is wrong: he has two singles wins in 1997, so still missing one singles win. Doubles wins says 27-20 but apparently should be 26-23. He was 0-3 doubles in 97. So total would be 27-23. I have too many wins(by one)! 
So I recorded one too many singles win and one too few doubles win...
Could it be that a singles win was accidentally marked as a doubles win??

For Earl, I am missing 3 singles wins, 3 singles losses, and 4 doubles wins. He played in 1996 and 1997. In 96 he was 1-3 singles, 1-0 doubles. In 97, he was 2-0 on singles and 3-0 doubles. So overall, he is missing 3-3 and 4-0. Thus his data adds up with the missing years. So Earl is good.

Deuel is correct. Hatch is correct.

Visually inspected all years and 2015 is the only year that is backwards.

In [250]:
playerinfo('Skyler Woodward','am')

['Skyler Woodward', 'am', 0, 3, '0-3', 0.0, 3, 3, '3-3', 50.0, '1-1', 4, 7, '4-7', 36.36]


Format                                    European_player  \
Year Match                                                               
2015 0        Teams  Karl Boyes, Niels Feijen, Albin Ouschan, Nick ...   
     3      Doubles                      Darren Appleton Albin Ouschan   
     5      Doubles                     Niels Feijen Nick van den Berg   
     8      Singles                                         Karl Boyes   
     12     Doubles                           Karl Boyes Albin Ouschan   
     17     Singles                                      Albin Ouschan   
2016 0        Teams  Niels Feijen, Albin Ouschan, Jayson Shaw, Mark...   
     3      Doubles                        Jayson Shaw Darren Appleton   
     5      Doubles                             Niels Feijen Mark Gray   
     10     Doubles                          Jayson Shaw Albin Ouschan   
     11     Singles                                      Albin Ouschan   

           European_score American_score  \
Year Match                                 
2015 0                  4              5   
     3                  1              5   
     5                  3              5   
     8                  5              3   
     12                 5              2   
     17                 5              4   
2016 0                  5              2   
     3                  1              5   
     5                  5              2   
     10                 5              3   
     11                 5              4   

                                              American_player Europe_won  
Year Match                                                                
2015 0      Corey Deuel, Mike Dechaine, Justin Bergman, Sh...      False  
     3                      Shane Van Boening Skyler Woodward      False  
     5                         Justin Bergman Skyler Woodward      False  
     8                                        Skyler Woodward       True  
     12                         Mike Dechaine Skyler Woodward       True  
     17                                       Skyler Woodward       True  
2016 0      Mike Dechaine, Shane Van Boening, Rodney Morri...       True  
     3                         Skyler Woodward Justin Bergman      False  
     5                      Shane Van Boening Skyler Woodward       True  
     10                         Rodney Morris Skyler Woodward       True  
     11                                       Skyler Woodward       True

# Rather than visual inspection, turn known data into unittests

## To do: add in triples/trebles data

trivia: only two players who didn't play any singles in their year but did play a team match: johnny archer, mark gray (2008)